In [12]:
import pandas as pd

In [20]:

transformed_users_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/staging/transformed/transformed_users.csv'
designations_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/staging/raw/designations.csv'
transformed_courses_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/staging/transformed/transformed_courses.csv'
transformed_progress_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/staging/transformed/transformed_progress.csv'


fact_users_file = 'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/mart/dimension/fact_users.csv'

transformed_users_df = pd.read_csv(transformed_users_file)
transformed_courses_df = pd.read_csv(transformed_courses_file)
designations_df = pd.read_csv(designations_file)
transformed_progress_df =  pd.read_csv(transformed_progress_file)

#print(transformed_users_df.dtypes)
#print(transformed_courses_df.dtypes)
#print(designations_df.dtypes)
 #.replace({'m': 'M', 'f': 'F'})
 #beg_comp_rate avg rate at which the user has completed a beginner level courses.
#interm_comp_rate avg rate at which the user has completed a intermediate level courses.
#adv_comp_rate avg rate at which the user has completed a advanced level courses.
#total beg courses completed by the user  Fill with 0 for users with no completed courses
#total interm courses completed by the user 
#total advanced courses completed by the user 
 #you can use any of the tranformed tables as well. 
 #['active'] true or false given if they are in any courses currently, (less than 100percent completed )
 #['learning_speed'] based on if they have completed more than 50% of their assigned courses  quicker than duration of the course

user_id                 int64
user_name              object
email                  object
role                   object
designation_id          int64
sex                    object
years_of_experience     int64
join_date              object
join_year               int64
tenure                  int64
join_month              int64
join_day                int64
dtype: object
user_id                        int64
title                         object
proficiency_level             object
chapters                       int64
duration                     float64
course_age                     int64
duration_category             object
proficiency_level_encoded      int64
created_day                    int64
created_month                  int64
created_year                   int64
dtype: object
id        int64
title    object
dtype: object


In [27]:
# Rename 'user_id' to 'course_id' if that’s the intended change
transformed_courses_df.rename(columns={'user_id': 'course_id'}, inplace=True)


In [30]:

transformed_progress_df.rename(columns={'id': 'progress_id'}, inplace=True)


In [21]:
# set all employees role as employee and drop the role column,
transformed_users_df['role'] = 'employee'
transformed_users_df.drop(columns=['role'], inplace=True)

In [38]:
merged_progress_df = pd.merge(transformed_progress_df, 
                               transformed_courses_df[['course_id', 'title', 'proficiency_level']], 
                               left_on='courseId', 
                               right_on='course_id', 
                               how='left')

In [39]:
transformed_users_df['sex'] = transformed_users_df['sex'].replace({'m': 'M', 'f': 'F'})


In [41]:
# avg completion rates
completion_rates = merged_progress_df.groupby('userId').agg(
    beg_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Beginner'].mean()),
    interm_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Intermediate'].mean()),
    adv_comp_rate=('percentage_completed', lambda x: x[merged_progress_df['proficiency_level'] == 'Advanced'].mean()),
    total_beg_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Beginner').sum()),
    total_interm_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Intermediate').sum()),
    total_adv_courses=('courseId', lambda x: (merged_progress_df['proficiency_level'] == 'Advanced').sum())
).reset_index()



In [43]:
fact_users_df = pd.merge(transformed_users_df, transformed_progress_df, left_on='user_id', right_on='userId', how='left')
fact_users_df = pd.merge(fact_users_df, transformed_courses_df[['course_id', 'proficiency_level']], left_on='courseId', right_on='course_id', how='left')

In [ ]:
fact_users_df['beg_comp_rate'] = (
    fact_users_df[fact_users_df['proficiency_level'] == 'Beginner']
    .groupby('user_id')['updated_day']  
    .mean()
)

fact_users_df['interm_comp_rate'] = (
    fact_users_df[fact_users_df['proficiency_level'] == 'Intermediate']
    .groupby('user_id')['updated_day']
    .mean()
)

fact_users_df['adv_comp_rate'] = (
    fact_users_df[fact_users_df['proficiency_level'] == 'Advanced']
    .groupby('user_id')['updated_day']
    .mean()
)


In [ ]:
# Save the final dimension table to CSV
fact_users_df.to_csv('C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/.venv/mart/dimension/fact_users.csv', index=False)

print("fact_users.csv created successfully created ")